In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
import malaya

/home/ubuntu/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
model = malaya.paraphrase.transformer(model = 't5')

2022-11-24 16:57:06.031188: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 16:57:06.096319: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 16:57:06.138634: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 16:57:06.140102: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [4]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [6]:
import json

test = []
with open('shuffled-test.json') as fopen:
    for l in fopen:
        test.append(json.loads(l))

In [7]:
from tqdm import tqdm

batch_size = 1

results = []
for i in tqdm(range(0, len(test), batch_size)):
    input_ids = [s['translation']['src'] for s in test[i:i + batch_size]]
    outputs = model.greedy_decoder(input_ids)
    results.extend(outputs)

  0%|                                                                                                       | 0/5544 [00:00<?, ?it/s]2022-11-24 16:57:24.047731: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 98697216 exceeds 10% of free system memory.
2022-11-24 16:57:24.087790: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 98697216 exceeds 10% of free system memory.
2022-11-24 16:57:24.327803: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 98697216 exceeds 10% of free system memory.
2022-11-24 16:57:24.327846: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 98697216 exceeds 10% of free system memory.
2022-11-24 16:57:24.367935: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 98697216 exceeds 10% of free system memory.
2022-11-24 16:57:32.472015: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
import feat

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5544/5544 [21:30<00:00,  4.30it/s]


In [8]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(test[no]['translation']['tgt'])

In [9]:
refs = [filtered_right]
sys = filtered_left
r = bleu.corpus_score(sys, refs)
r.__dict__

{'name': 'BLEU',
 'score': 35.86211127195363,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '61.1/40.6/31.3/25.2 (BP = 0.959 ratio = 0.960 hyp_len = 96986 ref_len = 101064)',
 'bp': 0.9588244215464514,
 'counts': [59234, 37096, 26876, 20285],
 'totals': [96986, 91442, 85898, 80355],
 'sys_len': 96986,
 'ref_len': 101064,
 'precisions': [61.074794300208275,
  40.56779160560793,
  31.288272136720295,
  25.24422873498849],
 'prec_str': '61.1/40.6/31.3/25.2',
 'ratio': 0.959649331116916}